# AAPL price forecasting
- This is an ipynb file created in colab environment.
- Runtime : GPU
- Wavenet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Github/Time-series_price_analysis')

In [ ]:
!ls /content/drive/MyDrive/Github/Time-series_price_analysis/*.py

/content/drive/MyDrive/Github/Time-series_price_analysis/make_dataset.py


## 0. Import Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random as rn
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

import keras.layers
from keras import optimizers
from keras.layers import Activation, Lambda, Input
from keras.layers import Conv1D, SpatialDropout1D, Flatten
from keras.layers import Convolution1D, Dense, Reshape
from keras.models import Input, Model
from typing import List, Tuple

import tensorflow as tf
import os

from make_dataset import time_transform, make_input

In [ ]:
# reproducibility
seed_num = 42
rn.seed(seed_num)
tf.random.set_seed(seed_num)
os.environ['PYTHONHASHSEED']=str(seed_num)

from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

## 1. Load and visualize dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Github/Time-series_price_analysis/dataset/AAPL.csv")

In [ ]:
df['Date'] = df['Date'].apply(lambda x: time_transform(x))
dataset = df.sort_values('Date').reset_index(drop=True)

In [ ]:
print("Shape of dataset :", dataset.shape)
dataset.head()

Shape of dataset : (2539, 7)


,Date,Price,Open,High,Low,Vol.,Change %
0,2012-02-27,18.78,18.62,18.87,18.44,547.58M,0.64%
1,2012-02-28,19.12,18.86,19.12,18.78,600.39M,1.81%
2,2012-02-29,19.37,19.34,19.56,19.13,952.00M,1.31%
3,2012-03-01,19.45,19.58,19.58,19.24,683.25M,0.41%
4,2012-03-02,19.47,19.44,19.53,19.38,431.71M,0.10%


In [ ]:
date = dataset['Date'].values
close = dataset['Price'].values

## 2. Preprocessing

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_close = scaler.fit_transform(np.array(close).reshape(-1,1))

In [ ]:
train_size = int(len(close)*0.795)
val_size = len(close)-train_size
train_data, val_data = scaled_close[0:train_size,:], scaled_close[train_size:len(close),:1]

In [ ]:
time_step = 1
X_train, y_train = make_input(train_data, time_step)
X_val, y_val = make_input(val_data, time_step)

In [ ]:
print("X train shape :", X_train.shape)
print("y train shape :", y_train.shape)
print("X val shape :", X_val.shape)
print("y val shape :", y_val.shape)

X train shape : (2016, 1)
y train shape : (2016,)
X val shape : (519, 1)
y val shape : (519,)


In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1] , 1)

print("X train shape :", X_train.shape)
print("X val shape :", X_val.shape)

X train shape : (2016, 1, 1)
X val shape : (519, 1, 1)


## 3. Modeling - Wavenet

In [ ]:
def gated_activation_units(x):
    tanh_out = Activation('tanh')(x)
    sig_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sig_out])


def residual_block(x, i, num_filters, kernel_size, padding):
    # i: The dilation power of 2


    prev_x = x
    conv = Conv1D(filters=num_filters, kernel_size = kernel_size, dilation_rate = i, padding = padding)(x) # dilated conv
    x = gated_activation_units(conv)  # gated activation units

    x = Convolution1D(num_filters, 1, padding='same')(x)  # skip connection

    res_x = keras.layers.add([prev_x, x])

    return res_x, x  # residual, skip connection

In [ ]:
# params
dilations = [1, 2, 4, 8, 16, 32]
num_filters = 64
padding = "causal"
kernel_size = 2
num_stacks = 1

In [ ]:
# wavenet
input = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = Convolution1D(num_filters, 1, padding = padding)(input)  # causal conv

skip_connections = []
for k in range(num_stacks):
    for i in dilations:
        x, skip_out = residual_block(x, i, num_filters, kernel_size, padding)  # residual and skip connection
        skip_connections.append(skip_out)

x = keras.layers.add(skip_connections) 

x = Activation('relu')(x)

# Since then, different from original Wavenet

x = Dense(16, activation="relu")(x)
# x = Dropout(0.1)(x)
output = Dense(1)(x)   


In [ ]:
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 1, 1)]       0           []                               
                                                                                                  
 conv1d_126 (Conv1D)            (None, 1, 64)        128         ['input_12[0][0]']               
                                                                                                  
 conv1d_127 (Conv1D)            (None, 1, 64)        8256        ['conv1d_126[0][0]']             
                                                                                                  
 activation_121 (Activation)    (None, 1, 64)        0           ['conv1d_127[0][0]']             
                                                                                            

In [ ]:
adam = Adam(learning_rate=0.001)
chk = ModelCheckpoint('/content/drive/MyDrive/Github/Time-series_price_analysis/model_weights/aapl/wavenet.pkl', monitor = 'val_loss', save_best_only = True, mode = 'min', verbose = 1)

model.compile(loss='mse', optimizer = adam)

In [ ]:
%%time
hist = model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=[chk], epochs=10, batch_size = 64, verbose = 1, shuffle = False)   # very fast

Epoch 1/30
31/32 [============================>.] - ETA: 0s - loss: 0.0010    
Epoch 1: val_loss improved from inf to 0.01028, saving model to /content/drive/MyDrive/Github/Time-series_price_analysis/model_weights/aapl/wavenet.pkl
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Github/Time-series_price_analysis/model_weights/aapl/wavenet.pkl/assets
32/32 [==============================] - 12s 209ms/step - loss: 0.0010 - val_loss: 0.0103
Epoch 2/30
31/32 [============================>.] - ETA: 0s - loss: 0.0052
Epoch 2: val_loss did not improve from 0.01028
32/32 [==============================] - 1s 32ms/step - loss: 0.0053 - val_loss: 0.0439
Epoch 3/30
31/32 [============================>.] - ETA: 0s - loss: 0.0038
Epoch 3: val_loss did not improve from 0.01028
32/32 [==============================] - 1s 28ms/step - loss: 0.0038 - val_loss: 0.0288
Epoch 4/30
31/32 [============================>.] - ETA: 0s - loss: 0.0057
Epoch 4: val_loss did not improve from 0.01028
32/32 [

## 4. Evaluation

In [ ]:
loaded_model = load_model("/content/drive/MyDrive/Github/Time-series_price_analysis/model_weights/aapl/wavenet.pkl")

In [ ]:
train_predicton = loaded_model.predict(X_train)
val_prediction = loaded_model.predict(X_val)

train_predicton = np.reshape(train_predicton, (train_predicton.shape[0], 1))
val_prediction = np.reshape(val_prediction, (val_prediction.shape[0], 1))

print("Train pred shape :", train_predicton.shape)
print("Val pred shape :", val_prediction.shape)

Train pred shape : (2016, 1)
Val pred shape : (519, 1)


In [ ]:
train_pred = scaler.inverse_transform(train_predicton)
val_pred = scaler.inverse_transform(val_prediction)
print(val_pred[:5])

[[76.32148 ]
 [73.7995  ]
 [72.798355]
 [66.9968  ]
 [71.83712 ]]


In [ ]:
close = np.array(close)
cc = close.reshape((len(close), 1))
tt1, tt2 = cc[0:train_size,:], cc[train_size:len(close),:]
xtrain, ytrain = make_input(tt1, 1)
Xval, yval = make_input(tt2, 1)

In [ ]:
val_rmse = math.sqrt(mean_squared_error(yval, val_pred))
print('Val RMSE: %.3f' % val_rmse)

Val RMSE: 2.779
